# 자동 미분

본 글에서는 JAX의 기초적인 자동미분 활용법에 대해서 알아보겠습니다. 마분값을 구하는 것은 현대 머신러닝에서 매우 중요한 부분입니다.

## 1. `jax.grad()`를 통해 기울기 구하기

JAX에서는 스칼라 값을 리턴하는 함수를 `jax.grad()` 변환함수를 통해 미분할 수 있습니다:

In [1]:
import jax
import jax.numpy as jnp
from jax import grad

grad_tanh = grad(jnp.tanh)
print(grad_tanh(2.0))

0.070650816


`jax.grad()`는 함수를 받고 함수를 내뱉습니다. 만약 파이썬으로 f라는 수학 함수를 구현했다면, `jax.grad(f)`는 $\nabla f$ 를 내뱉게 됩니다. 즉, `grad(f)(x)` 는 $\nabla f(x)$ 를 나타냅니다.

아래와 같이 `jax.grad()`를 여러번 중첩해서 씌울 수 있습니다:

In [3]:
print(grad(grad(jnp.tanh))(2.0))
print(grad(grad(grad(jnp.tanh)))(2.0))

-0.13621868
0.25265405


Jax의 자동미분 기능은 고계도함수를 쉽게 구할 수 있도록 해주는데, 함수를 미분한 결과가 곧 또다른 미분 가능한 함수의 형태로 나오기 때문입니다. 고로 고계도함수를 구한다는 건 그저 `jax.grad()` 를 쌓는것만큼 쉽습니다. 아래 변수가 하나인 식을 예로 확인해보겠습니다:

함수 $f(x) = x^3+2x^2-3x+1$은 아래와 같이 나타낼 수 있습니다:

In [4]:
f = lambda x: x**3 + 2*x**2 - 3*x + 1

dfdx = jax.grad(f)

함수 `f`의 고계도함수는:

$f'(x)=3x^2+4x-3$


$f''(x)=6x+4$


$f'''(x)=6$


$f^{iv}(x)=0$

이를 `jax.grad()`를 사용하면 파이썬에서도 매우 쉽게 구할 수 있습니다:

In [5]:
d2fdx = jax.grad(dfdx)
d3fdx = jax.grad(d2fdx)
d4fdx = jax.grad(d3fdx)

$x=1$일때의 해를 구할 때를 가정하면:

$f'(1)=4$

$f''(x1)=10$

$f'''(1)=6$

$f^{iv}(1)=0$

JAX를 사용했을 떄:

In [6]:
print(dfdx(1.))
print(d2fdx(1.))
print(d3fdx(1.))
print(d4fdx(1.))

4.0
10.0
6.0
0.0


## 2. 선형회귀 모델의 경사도 계산하기

아래 예제에서는 `jax.grad()`를 통해 선형회기 모델의 경사도를 구하는 방법을 살펴보겠습니다.

In [3]:
key = jax.random.key(0)

def sigmoid(x):
    return 0.5 * (jnp.tanh(x/2)+1)

def predict(W, b, inputs):
    return sigmoid(jnp.dot(inputs, W) + b)


# Build a toy dataset.
inputs = jnp.array([[0.52, 1.12,  0.77],
                    [0.88, -1.08, 0.15],
                    [0.52, 0.06, -1.30],
                    [0.74, -2.49, 1.39]])
targets = jnp.array([True, True, False, True])

def loss(W, b):
    preds = predict(W, b, inputs)
    label_probs = preds * targets + (1 - preds) * (1 - targets)
    return -jnp.sum(jnp.log(label_probs))

key, W_key, b_key = jax.random.split(key, 3)
W = jax.random.normal(W_key, (3,))
b = jax.random.normal(b_key, ())

`jax.grad()`를 통해 선형모델의 파라메터 `W`와 `b`에 대한 기울기 값을 구해봅시다. 이때 `jax.grad()`의 `argnums` 인자를 이용하여, 어떤 변수에 대한 기울기를 구하고 싶은지 지정할 수 있습니다. 예를 들어 `argnums=0`일 경우, 첫 번째 인자로 들어간 변수 `W`에 대한 기울기를 출력하는 식입니다. `argnums`의 기본값은 0이므로, `argnums`를 지정하지 않으면 첫 번째 인자에 대한 기울기를 구하게 됩니다.

In [4]:
W_grad = grad(loss, argnums=0)(W, b)
print(f'{W_grad}')

W_grad = grad(loss)(W, b)
print(f'{W_grad}')

b_grad = grad(loss, 1)(W, b)
print(f'{b_grad}')

W_grad, b_grad = grad(loss, (0, 1))(W, b)
print(f'{W_grad}')
print(f'{b_grad}')

[-0.16965583 -0.8774644  -1.4901346 ]
[-0.16965583 -0.8774644  -1.4901346 ]
-0.2922724485397339
[-0.16965583 -0.8774644  -1.4901346 ]
-0.2922724485397339


## 3. 중첩 리스트, 튜플, 딕셔너리 형태에 대해서 미분하기

Jax의 강력한 `pytree` 지원 덕에 중첩 리스트, 튜플, 딕셔너리 형태에 대해서도 미분을 할 수 있습니다:

In [5]:
def loss2(params_dict):
    preds = predict(params_dict['W'], params_dict['b'], inputs)
    label_probs = preds * targets + (1 - preds) * (1 - targets)
    return -jnp.sum(jnp.log(label_probs))

print(grad(loss2)({'W': W, 'b': b}))

{'W': Array([-0.16965583, -0.8774644 , -1.4901346 ], dtype=float32), 'b': Array(-0.29227245, dtype=float32)}


## 4. `jax.value_and_grad`를 사용하여 함수의 값과 기울기를 동시에 구하기

`jax.value_and_grad()`를 사용하면 함수의 값과 기울기를 동시에 구할 수 있습니다. 연산 또한 값과 기울기를 각각 구하는 것 보다 더욱 효율적입니다.

In [10]:
loss_value, Wb_grad = jax.value_and_grad(loss, (0, 1))(W, b)
print('loss value', loss_value)
print('loss value', loss(W, b))

loss value 3.0519388
loss value 3.0519388


5. 